# Projet 8: Participez à la conception d'une voiture autonome

## Segmentation des images complexe VGG16 FCN

### Importer les librairies necessaires

In [1]:
!pip install --upgrade tensorflow

     |████████████████████████████████| 458.3 MB 3.1 kB/s  eta 0:00:011
     |████████████████████████████████| 462 kB 57.8 MB/s eta 0:00:01
     |████████████████████████████████| 14.8 MB 44.4 MB/s eta 0:00:01
     |████████████████████████████████| 5.6 MB 64.2 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 61.8 MB/s eta 0:00:01
  Created wheel for clang: filename=clang-5.0-py3-none-any.whl size=30705 sha256=3720d1be8ede191a0af114edc9b6c1ca3f2526c0a31450418cda8666c88c93a3
  Stored in directory: /home/azureuser/.cache/pip/wheels/22/4c/94/0583f60c9c5b6024ed64f290cb2d43b06bb4f75577dc3c93a7
Successfully built clang
ERROR: pyldavis 3.3.1 requires sklearn, which is not installed.
ERROR: pandas-ml 0.6.1 requires enum34, which is not installed.
ERROR: torchvision 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.10.1 which is incompatible.
ERROR: torch-tb-profiler 0.3.1 has requirement pandas>=1.0.0, but you'll have pandas 0.25.3 which is incompatible.
ERROR: tensor

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Convolution2D,BatchNormalization,ReLU,LeakyReLU,Add,Activation
from tensorflow.keras.layers import GlobalAveragePooling2D,AveragePooling2D,UpSampling2D
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras import backend as K
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Activation, MaxPool2D, Concatenate
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.metrics import MeanIoU
from tensorflow.keras import Model
from tensorflow.keras.applications.vgg16 import VGG16

### Dèfinir mes fonctions pertes

In [3]:
def dice_coeff(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score

def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss

def total_loss(y_true, y_pred):
    loss = binary_crossentropy(y_true, y_pred) + (3*dice_loss(y_true, y_pred))
    return loss

### Dèfinir les catègories de mes masques

In [4]:
cats = {'void': [0, 1, 2, 3, 4, 5, 6],
 'flat': [7, 8, 9, 10],
 'construction': [11, 12, 13, 14, 15, 16],
 'object': [17, 18, 19, 20],
 'nature': [21, 22],
 'sky': [23],
 'human': [24, 25],
 'vehicle': [26, 27, 28, 29, 30, 31, 32, 33, -1]}

### Dèfinir les emplacements de mes images et masks

In [5]:
image_dir = 'aug_dataset/images'
mask_dir = 'aug_dataset/masks'
image_list = os.listdir(image_dir)
mask_list = os.listdir(mask_dir)
image_list.sort()
mask_list.sort()
print(f'Nbre Image: {len(image_list)}\nNbre Mask: {len(mask_list)}')

# sanity check
for i in range(len(image_list)):
    assert image_list[i] == mask_list[i]
print(image_list[2], mask_list[2])

Nbre Image: 5000
Nbre Mask: 5000
0003.jpg 0003.jpg


In [6]:
test_image_dir = 'data_f/image_val'
test_mask_dir = 'data_f/masque_val'
test_image_list = os.listdir(image_dir)
test_mask_list = os.listdir(mask_dir)
test_image_list.sort()
test_mask_list.sort()
print(f'Nbre Image test: {len(test_image_list)}\nNbre Mask test: {len(test_mask_list)}')

# sanity check
for i in range(len(image_list)):
    assert test_image_list[i] == test_mask_list[i]
print(test_image_list[2], test_mask_list[2])

Nbre Image test: 5000
Nbre Mask test: 5000
0003.jpg 0003.jpg


### Dèfinir les différents paramètres

In [7]:
batch_size = 16
samples = 5000
steps = samples//batch_size
img_height, img_width = 256,256
classes = 8
filters_n = 64

### Créer la class seg_gen

In [8]:
class seg_gen(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        idx = np.random.randint(0, samples, batch_size)
        batch_x, batch_y = [], []
        for i in idx:
            _image = image.img_to_array(image.load_img(f'{image_dir}/{image_list[i]}', target_size=(img_height, img_width)))/255.   
            img = image.img_to_array(image.load_img(f'{mask_dir}/{mask_list[i]}', grayscale=True, target_size=(img_height, img_width)))
            img = np.squeeze(img)
            mask = np.zeros((img.shape[0], img.shape[1], 8))
            for i in range(-1, 34):
                if i in cats['void']:
                    mask[:,:,0] = np.logical_or(mask[:,:,0],(img==i))
                elif i in cats['flat']:
                    mask[:,:,1] = np.logical_or(mask[:,:,1],(img==i))
                elif i in cats['construction']:
                    mask[:,:,2] = np.logical_or(mask[:,:,2],(img==i))
                elif i in cats['object']:
                    mask[:,:,3] = np.logical_or(mask[:,:,3],(img==i))
                elif i in cats['nature']:
                    mask[:,:,4] = np.logical_or(mask[:,:,4],(img==i))
                elif i in cats['sky']:
                    mask[:,:,5] = np.logical_or(mask[:,:,5],(img==i))
                elif i in cats['human']:
                    mask[:,:,6] = np.logical_or(mask[:,:,6],(img==i))
                elif i in cats['vehicle']:
                    mask[:,:,7] = np.logical_or(mask[:,:,7],(img==i))
            mask = np.resize(mask,(img_height,img_width, 8))
            batch_x.append(_image)
            batch_y.append(mask)
            X = np.array(batch_x)
            Y = np.array(batch_y)
        return X, Y

In [9]:
train_gen = seg_gen(image_list, mask_list, batch_size)

In [10]:
for X, Y in train_gen:
    break
X.shape, Y.shape

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/keras_preprocessing/image/utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


((16, 256, 256, 3), (16, 256, 256, 8))

### Dèfinir le modèle VGG16FCN8

### Modèle de segmentation - FCN+Transfer Learning

In [11]:
def fcn(vgg, classes = classes, fcn8 = False, fcn16 = False):
    pool5 = vgg.get_layer('block5_pool').output 
    pool4 = vgg.get_layer('block4_pool').output
    pool3 = vgg.get_layer('block3_pool').output
    
    conv_6 = Conv2D(1024, (7, 7), activation='relu', padding='same', name="conv_6")(pool5)
    conv_7 = Conv2D(1024, (1, 1), activation='relu', padding='same', name="conv_7")(conv_6)
    
    conv_8 = Conv2D(classes, (1, 1), activation='relu', padding='same', name="conv_8")(pool4)
    conv_9 = Conv2D(classes, (1, 1), activation='relu', padding='same', name="conv_9")(pool3)
    
    deconv_7 = Conv2DTranspose(classes, kernel_size=(2,2), strides=(2,2))(conv_7)
    add_1 = Add()([deconv_7, conv_8])
    deconv_8 = Conv2DTranspose(classes, kernel_size=(2,2), strides=(2,2))(add_1)
    add_2 = Add()([deconv_8, conv_9])
    deconv_9 = Conv2DTranspose(classes, kernel_size=(8,8), strides=(8,8))(add_2)
    
    if fcn8 :
        output_layer = Activation('softmax')(deconv_9)
    elif fcn16 :
        deconv_10 = Conv2DTranspose(classes, kernel_size=(16,16), strides=(16,16))(add_1)
        output_layer = Activation('softmax')(deconv_10)
    else :
        deconv_11 = Conv2DTranspose(classes, kernel_size=(32,32), strides=(32,32))(conv_7)
        output_layer = Activation('softmax')(deconv_11)
    
    model = Model(inputs=vgg.input, outputs=output_layer)
    return model

In [12]:
callbacks = [
    keras.callbacks.ModelCheckpoint("model_VGG16FCN_check.h5", save_best_only=True),
    keras.callbacks.EarlyStopping(patience=3, verbose=1)
]

In [13]:
vgg = VGG16(include_top=False, weights='imagenet', input_shape=(img_height, img_width, 3))

58900480/58889256 [==============================] - 0s 0us/step


In [14]:
model = fcn(vgg, fcn8=True)
model.compile(
    'Adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 256, 256, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv2 (Conv2D)          (None, 256, 256, 64  36928       ['block1_conv1[0][0]']           
                                )                                                             

In [15]:
model.fit_generator(train_gen, steps_per_epoch=steps, epochs=8, callbacks=callbacks, workers=6)

<ipython-input-15-22f8966706a9>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_gen, steps_per_epoch=steps, epochs=8, callbacks=callbacks, workers=6)


Epoch 1/8
312/312 [==============================] - 228s 557ms/step - loss: 1.5772 - accuracy: 0.4160
Epoch 2/8
312/312 [==============================] - 174s 552ms/step - loss: 1.3159 - accuracy: 0.4925
Epoch 3/8
312/312 [==============================] - 175s 555ms/step - loss: 1.1907 - accuracy: 0.5447
Epoch 4/8
312/312 [==============================] - 174s 549ms/step - loss: 1.1002 - accuracy: 0.5782
Epoch 5/8
312/312 [==============================] - 174s 551ms/step - loss: 1.0532 - accuracy: 0.5964
Epoch 6/8
312/312 [==============================] - 174s 551ms/step - loss: 1.0070 - accuracy: 0.6126
Epoch 7/8
312/312 [==============================] - 175s 554ms/step - loss: 0.9776 - accuracy: 0.6222
Epoch 8/8
312/312 [==============================] - 172s 547ms/step - loss: 0.9521 - accuracy: 0.6302


In [16]:
model.save('model_VGG16FCN.h5')